<a href="https://colab.research.google.com/github/bhaveshsingh0206/Text-Summmariser./blob/master/text_summariser_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Loading file from Drive
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# https://drive.google.com/file/d/1ghQHoCxCEqrq_RUiHkukps-gsZwsH__A/view?usp=sharing

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:

# downloaded = drive.CreateFile({'id':"1ghQHoCxCEqrq_RUiHkukps-gsZwsH__A"})   
# downloaded.GetContentFile('final_model.pt') 
downloaded = drive.CreateFile({'id':"1FfO8fsNXnenTfsq7skCgZpv6PBBQhjpa"})   
downloaded.GetContentFile('glove.6B.100d.txt')  

In [0]:
downloaded = drive.CreateFile({'id':"13JdWTJ4hSKpUQnOlAxmzTdjNVUjco4fV"}) 
downloaded.GetContentFile('DF.csv') 

In [0]:
word2Vec = {}
with open('glove.6B.100d.txt') as f:
  for line in f:
    line = line.rstrip().split(' ')
    word = line[0]
    embedding_vector = line[1:]
    word2Vec[word] = embedding_vector

In [6]:
import pandas as pd
dataset = pd.read_csv('DF.csv')
dataset.head()

,headlines,article
0,"India reports more than 5,000 coronavirus case...",The total number of coronavirus cases in India...
1,"Because of few jokers, COVID-19 is spreading: ...",Salman Khan has condemned the attack on medica...
2,"45% districts without a COVID-19 case, 27 dist...",The government on Thursday announced that 45% ...
3,68-yr-old UP man suffering from cold declared ...,"A 68-year-old man from Uttar Pradesh's Amroha,..."
4,"Lockdown is like pause button, will not defeat...","Addressing the media via video on Thursday, Co..."


In [0]:
SOS_token = '<start>'
EOS_token = '<end>'
PAD_token = '<pad>'
UNK_token = '<unk>'

SOS_idx = 0
EOS_idx = 1
UNK_idx = 3
PAD_idx = 2

In [0]:
class Corpus:
  def __init__(self):
    self.index2word = {
        0: SOS_token,
        1: EOS_token,
        2: PAD_idx,
        3: UNK_token
    }
    self.indices = 4
    self.word2index = {word:k for k,word in self.index2word.items()}

  def index_words(self, sent):
    for word in sent:
      self.index_word(word)

  def index_word(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.indices
      self.index2word[self.indices] = word
      self.indices += 1
  
  def unidex_words(self, indices):
      return [self.index2word[index] for index in indices]

In [0]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
tokenizer = Tokenizer(nlp.vocab)
toknizers = {
    'hin': "",
    'eng': tokenizer
}

In [0]:
def prepareCorpus(min_count, tokenizer, sents, language = 'eng'):
  freq_words = {}
  sents_tokenized = []
  for sent in sents:
    if language == 'eng':
      sent_tokenized = [w.text.lower() for w in tokenizer(sent)]
    else :
      sent_tokenized = [w for w in sent.split(' ')]
    sents_tokenized.append(sent_tokenized)
    for word in sent_tokenized:
      if word not in freq_words:
        freq_words[word] = 1
      else:
        freq_words[word] += 1
  # print(freq_words)
  for i, sent_tokenized in enumerate(sents_tokenized):
    sent_tokenized = [t if freq_words[t] >= min_count  else UNK_token for t in sent_tokenized]
    sents_tokenized[i] = sent_tokenized
  return sents_tokenized

In [0]:
def read_corpus(sents):
  corpus = Corpus()
  for sent in sents:
    corpus.index_words(sent)
  return corpus

In [0]:
source_sequences = prepareCorpus(1, tokenizer=toknizers['eng'], sents=dataset['article'])
target_sequences = prepareCorpus(1, tokenizer=toknizers['eng'], sents=dataset['headlines'])

In [13]:
print(target_sequences[0])

['india', 'reports', 'more', 'than', '5,000', 'coronavirus', 'cases', 'in', '5', 'days,', 'total', 'cases', 'rise', 'to', '12,759']


In [14]:
dataset['headlines'][0]

'India reports more than 5,000 coronavirus cases in 5 days, total cases rise to 12,759'

In [0]:
source_vocab = read_corpus(source_sequences)
target_vocab = read_corpus(target_sequences)

In [16]:
print(len(target_vocab.word2index))

57772


In [0]:
DIMENSIONS = 100

In [0]:
import numpy as np
num_words = len(target_vocab.word2index)
out_of_vocab = []
target_embedding_matrix = np.zeros((num_words, 100))
for j,k in target_vocab.word2index.items():
  if k < num_words:
    embedding_vector = word2Vec.get(j)
    if embedding_vector is not None:
      target_embedding_matrix[k] = embedding_vector
    else:
      out_of_vocab.append(j)


In [19]:
print(out_of_vocab)

['<start>', '<end>', 2, '<unk>', 'days,', 'jokers,', 'covid-19', 'spreading:', '45%', 'case,', 'days:', '68-yr-old', '+ve', 'button,', 'virus:', 'ex-commando,', 'attacks,', "india's", 'pandemic:', 'facility:', 'place,', 'yrs,', 'safety:', 'who,', 'india,', 'zomato', 'guj', 'case;', "can't", 'less:', "'in", "eye'", 'storm:', "man's", 'disease:', 'conferencing,', '\xa0', 'govt;', "we'll", 'thing:', "it'll", 'harjeet', 'nihangs', "kerala's", '0.5%', "world's", '5%:', '19-yr-old', '+ve,', 'non-hotspot', 'areas,', 'districts?', "uttarakhand's", "'food", "vans'", 'homeless,', 'covid-19:', '550-bed', "up's", '22:', 'together:', 'chairman,', '30%', 'agra,', 'karnataka;', 'rajasthan,', "state's", 'colour:', "'misguided'", 'family:', "son's", 'bottle,', 'night,', 'cases,', '1,578;', 'lockdown?', '650;', 'hospital;', 'house,', 'cctv:', "didn't", 'lockdown,', 'hospitals:', "pradesh's", "patient's", 'transmission,', 'outbreaks:', "hasn't", 'districts,', 'crucial:', 'guwahati,', 'himanta', 'up,', '1

In [0]:
import numpy as np
num_words = len(source_vocab.word2index)
out_of_vocab = []
source_embedding_matrix = np.zeros((num_words, 100))
for j,k in source_vocab.word2index.items():
  if k < num_words:
    embedding_vector = word2Vec.get(j)
    if embedding_vector is not None:
      source_embedding_matrix[k] = embedding_vector
    else:
      out_of_vocab.append(j)

In [0]:
RANDOM_SEED = 10
np.random.seed(RANDOM_SEED)

source_length = len(source_sequences)
indices = np.random.permutation(source_length)


training_indices = indices[:int(source_length)]
val_indices = indices[int(source_length):]

training_source = [source_sequences[i] for i in training_indices]


training_target = [target_sequences[i] for i in training_indices]


In [0]:
def indexed_from_setence(vocab, sent):
  return [vocab.word2index[w] for w in sent]

In [0]:
def tensor_from_sentence(vocab, sent, max_seq_length):
  indexes = indexed_from_setence(vocab, sent)
  indexes.append(EOS_idx)
  indexes.insert(0,SOS_idx)
  if len(indexes) < max_seq_length:
    indexes += [PAD_idx] * (max_seq_length - len(indexes))
  tensor = torch.LongTensor(indexes)
  return tensor

In [0]:
def tensor_from_pair(source_sequences, target_sequences, max_source_seq_length, max_target_seq_length):
  source_tensor = tensor_from_sentence(source_vocab, source_sequences, max_source_seq_length).unsqueeze(1)
  target_tensor = tensor_from_sentence(target_vocab, target_sequences, max_target_seq_length).unsqueeze(1)
  return (source_tensor, target_tensor)

In [25]:
max_source_seq_length = max(len(s) for s in source_sequences) + 2
max_target_seq_length = max(len(s) for s in target_sequences) + 2

print(max_source_seq_length)
print(max_target_seq_length)

83
22


In [0]:
training_set = []
for source, target in zip(training_source, training_target):
  # print(source)
  training_set.append(tensor_from_pair(source, target, max_source_seq_length, max_target_seq_length))

X,y = zip(*training_set)
X = torch.transpose(torch.cat(X, dim=-1), 1, 0)
y = torch.transpose(torch.cat(y, dim=-1), 1, 0)

In [29]:
X.size()

torch.Size([64094, 83])

In [30]:
y[0]

tensor([    0,   296,   163,  1281,  1954,    16, 10438, 40905,    11,  2397,
           74,  3643,     1,     2,     2,     2,     2,     2,     2,     2,
            2,     2])

In [0]:
import math
def batch_generator(batch_indices, batch_size):
    batches = math.ceil(len(batch_indices)/batch_size)
    for i in range(batches):
        batch_start = i*batch_size
        batch_end = (i+1)*batch_size
        if batch_end > len(batch_indices):
            yield batch_indices[batch_start:]
        else:
            yield batch_indices[batch_start:batch_end]

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [0]:
class EncoderModel(nn.Module):
  def __init__(self, dimension, encoder_lstm_units, num_layers, input_vocab_size):
    super(EncoderModel, self).__init__()
    self.encoder_lstm_units = encoder_lstm_units
    self.num_layers = num_layers
    self.dimension = dimension
    self.embedding = nn.Embedding(input_vocab_size, dimension)
    self.embedding.weights = torch.nn.Parameter(torch.from_numpy(source_embedding_matrix).to(device))
    self.embedding.weight.requires_grad = True 
    # print("Encoder lstm")
    self.lstm = nn.LSTM(dimension, self.encoder_lstm_units, num_layers=num_layers, batch_first = True,bidirectional = True)
    # print("Encoder lstm cleared")
    # print(self.lstm)
  
  def forward(self, x, hidden):
    # print("i m here in forward")
    # print("initial_hidden_state ",hidden.size())
    # print(x.size())
    x = self.embedding(x)
    # print("Embedding size in encoder ", x.size())
    enoder_outputs, enoder_states = self.lstm(x, hidden)
    # print("i m exiting forward")
    # print("enoder_outputs size in encoder ", enoder_outputs.size())
    return enoder_outputs, enoder_states
    
  

  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers*2, batch_size, self.encoder_lstm_units).to(device)
    c = torch.zeros(self.num_layers*2, batch_size, self.encoder_lstm_units).to(device)

    # print("hidden_states_in_size in encoder",hidden.size())
    return (hidden,c)

In [0]:
# Birectional - lstm_units/2
class AttentionModel(nn.Module):
  def __init__(self, lstm_unit):
    super(AttentionModel, self).__init__()
    self.dense1 = nn.Linear(4*lstm_unit, lstm_unit)
    self.tanh = nn.Tanh()
    self.dense2 = nn.Linear(lstm_unit, 1)
    self.softmax = nn.Softmax(dim=1)
  def forward(self, encoder_outputs, decoder_input):
    # encoder_outputs   # batch,input_seq_len,lstm_unit
    # decoder_input     # batch,1,lstm_unit
    # print("decoder_input ", decoder_input.size())
    decoder_input = decoder_input.reshape(decoder_input.size(1), decoder_input.size(0), decoder_input.size(2))
    # print("decoder_input ", decoder_input.size())
    decoder_input = decoder_input.view(decoder_input.size(0),1,-1)
    # print("new decoder_input ", decoder_input.size())
    # print("encoder_outputs ",encoder_outputs.size())
    modified_decoder_input = decoder_input.repeat(1, (encoder_outputs.size(1)), 1)
    # print("modified_decoder_input ",modified_decoder_input.size())
    concatenated = torch.cat((encoder_outputs,modified_decoder_input),dim=2) #batch,input_seq_len,2*lstm_unit
    # print("concatenated ",concatenated.size())
    # print("concatenated")
    dense1 = self.dense1(concatenated)               # batch,input_seq_len, lstm_unit
    dense1 = self.tanh(dense1)
    # print("dense1")

    dense2 = self.dense2(dense1)      #batch, input_seq_len, 1
    # print("dense2 ", dense2.size())
    # print("")
    alphas = self.softmax(dense2)
    # print("alphas ",alphas.size())
    # print("alphas")
    alphas = alphas.reshape(alphas.size(0), alphas.size(2), alphas.size(1))
    context = torch.bmm(alphas, encoder_outputs)  # batch, input_seq_len, 1 * batch,input_seq_len,lstm_unit = batch, 1, lstm_unit
    return context, alphas
  

In [0]:
class DecoderModel(nn.Module):
  def __init__(self, dimension, decoder_lstm_units, num_layers, output_vocab_size, attentionModel):
    super(DecoderModel, self).__init__()
    self.decoder_lstm_units = decoder_lstm_units
    self.num_layers = num_layers
    self.dimension = dimension
    self.attentionModel = attentionModel
    self.embedding = nn.Embedding(output_vocab_size, dimension)
    self.embedding.weights = torch.nn.Parameter(torch.from_numpy(target_embedding_matrix).to(device))
    self.embedding.weight.requires_grad = True 

    self.lstm = nn.LSTM((dimension+decoder_lstm_units*2), decoder_lstm_units, num_layers = num_layers,batch_first = True)
    # print("decoder lstm cleared")

  def train(self, x, initial_hidden_state, encoder_output):
    # print("Yes boy i m trying attention")
    context, alphas = self.attentionModel(encoder_output, initial_hidden_state[0]) #context -> batch,1,lstm_units
    # print("alphas have been calculated anf even context has been returned")
    x = self.embedding(x).unsqueeze(1) #batch, 1, dimensions
    context_concatenated = torch.cat((x, context), dim=2) #batch,1,dimensions+lstm_units
    
    decoder_output, decoder_states = self.lstm(context_concatenated, initial_hidden_state)
    return decoder_output, decoder_states
  

  def init_hidden(self,batch_size):
    hidden = torch.zeros(self.num_layers*1, batch_size, self.decoder_lstm_units).to(device)
    c = torch.zeros(self.num_layers*1, batch_size, self.decoder_lstm_units).to(device)
    # print(type(hidden))
    return (hidden,c)   

In [0]:
class Seq2Seq(nn.Module):
  
  def __init__(self, dimension, lstm_units, num_layers, output_vocab_size, input_vocab_size):
    super(Seq2Seq, self).__init__()
    self.encoder = EncoderModel(dimension, lstm_units, num_layers, input_vocab_size)
    self.attention = AttentionModel(lstm_units)
    self.decoder = DecoderModel(dimension, lstm_units, num_layers, output_vocab_size, self.attention)
    self.dense = nn.Linear(lstm_units, output_vocab_size)
    self.softmax = nn.Softmax()
  
  def encoder_forward(self, x):
    batch_size = x.shape[0]
    encoder_initial_state = self.encoder.init_hidden(batch_size)
    # print("encoder_initial_state ",encoder_initial_state.size())
    encoder_output, enoder_hidden_states = self.encoder(x, encoder_initial_state)
    # print("Exiting encoder_forward")
    return encoder_output
  
  def forward_train(self, x, y):
    # print("do i even exist?")
    batch_size = x.shape[0]
    encoder_output = self.encoder_forward(x)
    # print("do i even exist? part 2")
    decoder_states = self.decoder.init_hidden(batch_size)
    H = []
    for i in range(y.shape[1]):
      decoder_input_single = y[:,i]
      # print(decoder_input_single.size())
      if i != 0:
        z = random.random()
        # print(z)
        if z > 0.0:
          decoder_input_single = y_predicted
          # print("Second",decoder_input_single.size())

        else :
          decoder_input_single = y[:,i]
      decoder_output, decoder_states = self.decoder.train(decoder_input_single, decoder_states, encoder_output) #batch, 1, lstm_units
      h = self.dense(decoder_output.squeeze(1)) #batch,output_vocab_size
      H.append(h.unsqueeze(2))
      y_predicted = self.softmax(h)
      _, y_predicted = torch.max(y_predicted, dim=1)
    return torch.cat(H, dim=2)
  

  def predict(self, x):
    batch_size = x.shape[0]
    encoder_output = self.encoder_forward(x)
    # print("got encoder_output")
    decoder_states = self.decoder.init_hidden(batch_size)
    # print("Got decoder_states")
    current_y = SOS_idx
    result = [current_y]
    counter = 0
    while current_y != EOS_idx and counter<=20:
      # print("I m hee boys ",counter)
      inpu = torch.tensor([current_y]).to(device)
      # print(inpu)
      decoder_output, decoder_hidden_states = self.decoder.train(inpu, decoder_states, encoder_output)
      
      # print("Before passing to dense wile testing ",decoder_output.size())
      h = self.dense(decoder_output.squeeze(1))
      # print("h before squeezing ",h.size())
      h = h.squeeze(0)
      # print("After passing to dense wile testing ",h.size())
      y = self.softmax(h)
      _, current_y = torch.max(y, dim=0)
      # print(current_y)
      # print(type(current_y))
      current_y = current_y.item()
      result.append(current_y)
      counter += 1
    return result

In [146]:
from torch.optim import Adam
model = Seq2Seq(dimension=100, lstm_units=256, num_layers=2, output_vocab_size=len(target_vocab.word2index), input_vocab_size=len(source_vocab.word2index))
optim = Adam(model.parameters(), lr=0.001)
cross_entropy = nn.CrossEntropyLoss()
model.to(device)

Seq2Seq(
  (encoder): EncoderModel(
    (embedding): Embedding(189207, 100)
    (lstm): LSTM(100, 256, num_layers=2, batch_first=True, bidirectional=True)
  )
  (attention): AttentionModel(
    (dense1): Linear(in_features=1024, out_features=256, bias=True)
    (tanh): Tanh()
    (dense2): Linear(in_features=256, out_features=1, bias=True)
    (softmax): Softmax(dim=1)
  )
  (decoder): DecoderModel(
    (attentionModel): AttentionModel(
      (dense1): Linear(in_features=1024, out_features=256, bias=True)
      (tanh): Tanh()
      (dense2): Linear(in_features=256, out_features=1, bias=True)
      (softmax): Softmax(dim=1)
    )
    (embedding): Embedding(57772, 100)
    (lstm): LSTM(612, 256, num_layers=2, batch_first=True)
  )
  (dense): Linear(in_features=256, out_features=57772, bias=True)
  (softmax): Softmax(dim=None)
)

In [139]:
# model.load_state_dict(torch.load('final_model.pt'))
# model.eval()
dataset.count

<bound method DataFrame.count of                                                headlines                                            article
0      India reports more than 5,000 coronavirus case...  The total number of coronavirus cases in India...
1      Because of few jokers, COVID-19 is spreading: ...  Salman Khan has condemned the attack on medica...
2      45% districts without a COVID-19 case, 27 dist...  The government on Thursday announced that 45% ...
3      68-yr-old UP man suffering from cold declared ...  A 68-year-old man from Uttar Pradesh's Amroha,...
4      Lockdown is like pause button, will not defeat...  Addressing the media via video on Thursday, Co...
...                                                  ...                                                ...
64089   Lamborghini Huracan LP580-2 launched at ₹2.99 cr  Italian luxury car maker Lamborghini on Friday...
64090           'Top Gear' set to return to BBC on May 8  Automobile TV series 'Top Gear' will return t

In [0]:
from tqdm import tqdm_notebook as tqdm

BATCH_SIZE = 64
total_batches = int(len(X)/BATCH_SIZE) + 1
indices = list(range(len(X)))

for epoch in range(35):
    # Training
    total_loss = 0.0
    for step, batch in tqdm(enumerate(batch_generator(indices, BATCH_SIZE)),
                            desc='Training epoch {}'.format(epoch+1),
                            total=total_batches):
      
        x = X[batch, :].to(device)
        # y for teacher forcing is all sequence without a last element
        y_tf = y[batch, :-1].to(device)
        # y for loss calculation is all sequence without a last element
        y_true = y[batch, 1:].to(device)
        # (batch_size, vocab_size, seq_length)
        H = model.forward_train(x, y_tf)
        # print(H.shape)
        # print(y_true.size())
        loss = cross_entropy(H, y_true)

        assert loss.item() > 0

        optim.zero_grad()
        loss.backward()
        optim.step()
        total_loss += loss.item()
        

    print('Epoch {} training is finished, loss: {:.4f}'.format(epoch+1, total_loss/total_batches))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



Epoch 1 training is finished, loss: 4.5758



Epoch 2 training is finished, loss: 4.2988


In [100]:
alphas = torch.zeros(64, 83, 1)
alphas = F.softmax(alphas, dim=1)
print(alphas.size())

torch.Size([64, 83, 1])


In [123]:
model.parameters

<bound method Module.parameters of Seq2Seq(
  (encoder): EncoderModel(
    (embedding): Embedding(189207, 100)
    (lstm): LSTM(100, 256, num_layers=2, batch_first=True, bidirectional=True)
  )
  (attention): AttentionModel(
    (dense1): Linear(in_features=1024, out_features=256, bias=True)
    (tanh): Tanh()
    (dense2): Linear(in_features=256, out_features=1, bias=True)
    (softmax): Softmax(dim=None)
  )
  (decoder): DecoderModel(
    (attentionModel): AttentionModel(
      (dense1): Linear(in_features=1024, out_features=256, bias=True)
      (tanh): Tanh()
      (dense2): Linear(in_features=256, out_features=1, bias=True)
      (softmax): Softmax(dim=None)
    )
    (embedding): Embedding(57772, 100)
    (lstm): LSTM(612, 256, num_layers=2, batch_first=True)
  )
  (dense): Linear(in_features=256, out_features=57772, bias=True)
  (softmax): Softmax(dim=None)
)>

In [0]:
import random

In [0]:
# torch.save(model.state_dict(),'finalmodel.pt')

In [0]:
# from google.colab import files
# files.download('finalmodel.pt')

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# model_save_name = 'final_model.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}" 
# torch.save(model.state_dict(), path)

In [0]:
model

Seq2Seq(
  (encoder): EncoderModel(
    (embedding): Embedding(189207, 100)
    (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  )
  (attention): AttentionModel(
    (dense1): Linear(in_features=512, out_features=256, bias=True)
    (tanh): Tanh()
    (dense2): Linear(in_features=256, out_features=1, bias=True)
    (softmax): Softmax(dim=None)
  )
  (decoder): DecoderModel(
    (attentionModel): AttentionModel(
      (dense1): Linear(in_features=512, out_features=256, bias=True)
      (tanh): Tanh()
      (dense2): Linear(in_features=256, out_features=1, bias=True)
      (softmax): Softmax(dim=None)
    )
    (embedding): Embedding(57772, 100)
    (lstm): LSTM(356, 256, batch_first=True)
  )
  (dense): Linear(in_features=256, out_features=57772, bias=True)
  (softmax): Softmax(dim=None)
)

In [0]:
for i,x in enumerate(source_sequences):
  if i<50005 and i > 50000:
    print("Article is : ")
    # print(' '.join(x))
    print("----------------------------------------")
    a = tensor_from_sentence(source_vocab, x, max_source_seq_length)
    # print(a.size())
    y = model.predict(a.to(device).unsqueeze(0))
    headline = ' '.join(target_vocab.unidex_words(y[1:-1]))
    print("Original headline : ")
    print(dataset['article'][i])
    print("My headline : ")
    print(headline)
    print("----------------------------------------")

Article is : 
----------------------------------------
Original headline : 
As per a study by the University of British Columbia, if unchecked, global warming will cause irreversible damage to marine life in the oceans. This, in turn, will force fish to search for cooler waters, destroying valuable coral reefs. These adverse effects of global warming can be prevented by keeping global average temperatures within 2 degrees Celsius above pre-industrial temperatures. 
My headline : 
marine countries: 2018, 29,000 2018, 29,000 2018, 29,000 2018, 29,000 2018, 29,000 2018, 29,000 2018, 29,000 2018, 29,000 2018, 29,000
----------------------------------------
Article is : 
----------------------------------------
Original headline : 
European Space Agency's spacecraft Philae that landed on a comet has found some organic material on its surface, indicating an abundance of alien microbial life. As per reports, the organic-rich black crust of the comet, named 67P/Churyumov-Gerasimenko, was likel

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Original headline : 
The trailer of 'He Named Me Malala', a documentary on Nobel Peace Prize winner Malala Yousafzai, has been unveiled on YouTube. The documentary, directed by Davis Guggenheim, offers a look into Malala's life before and after the Taliban attack wherein she was severely wounded by a gunshot. The film underscores the importance and influence of Malala's family throughout her journey.
My headline : 
trailer world world world world world world world world world world world world world world world world world world world
----------------------------------------


In [0]:
max_source_seq_length

83

In [0]:
import random
random.random()

0.2478203541027495

In [0]:
a = torch.zeros(2,2,12)

In [60]:
a.view(2,1,-1).size()

torch.Size([2, 1, 24])